In [1]:
# Passo 1: Carregar o dataset
import pandas as pd

dados = pd.read_csv("Historico_de_materias.csv")


In [2]:
# Para exibir somente os tipos de assunto
valores_unicos = dados['assunto'].unique()

print("Valores únicos na coluna 'assunto':")
print(valores_unicos)

Valores únicos na coluna 'assunto':
['economia' 'esportes' 'politica' 'famosos' 'tecnologia']


In [3]:
# Passo 2: Criar um novo dataframe somente para o assunto 'esporte'
dados_esportes = dados[dados['assunto'] == 'esportes']


In [ ]:
dados_esportes.head()

In [4]:
# Passo 3: Carregar spaCy
# Biblioteca de processamento de linguagem natural para tarefas como tokenização, 
# lematização entre outras.
import spacy

# Modelo pré-treinado otimizado para a língua portuguesa
nlp = spacy.load("pt_core_news_sm")


In [5]:
# Passo 4: Configurar palavras de parada

# Essa biblioteca oferece ferramentas para análise de texto, como stemmers (radicalizadores),
#  stopwords, tokenizadores, e entre outras funções
import nltk

# Importa o conjunto de palavras de parada
from nltk.corpus import stopwords

# Verificar stopwords
# lista padrão da biblioteca pré-definida de palavras irrelevantes
nltk.download('stopwords')

# Carrega as stopwords em português do NLTK e as converte em um conjunto Python
api_stop_words = set(stopwords.words('portuguese'))
# Incluir vogais isoladas não parece contribuir para o significado.
minhas_stop_words = {'a', 'e', 'i', 'o', 'u', 'né', 'aí', 'tá', 'então'}
# Combinação das palavras de parada padrão do NLTK com "minhas_stop_words" 
stop_words = api_stop_words | minhas_stop_words


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Daniel\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
# Passo 5: Função de tratamento de texto

# Biblioteca padrão do Python para expressões regulares
import re

# Expressões regulares permitem manipular e encontrar padrões em strings, 
# como remover caracteres indesejados ou verificar formatos específicos (ex.: datas, e-mails).
# Para realizar a limpeza do texto removendo caracteres que não sejam letras ou espaços.

# função usada para pré-processar textos antes de realizar análises mais avançadas.
def tratamento_pln(texto):
    # Converte todo o texto para letras minúsculas
    texto = texto.lower()
    # Substitui todos os caracteres indesejados por uma string vazia
    texto = re.sub(r'[^a-zA-Záéíóú\s]', '', texto)
    # Lematiza as palavras de acordo com o pipeline da spacy e divide o texto em tokens
    doc = nlp(texto)
    # Cria uma lista de palavras tratadas que não são pontuações nem stopwords
    clean_tokens = [token.lemma_ for token in doc if token.text not in stop_words and not token.is_punct]
    # Combina a lista de palavras limpas (clean_tokens) em uma única string
    clean_text = ' '.join(clean_tokens)
    return clean_text


In [94]:
# Visualizando as Stop Words
print("Tamanho do conjunto stop_words:",len(stop_words),"\nStop_words ordenadas: \n",sorted(list(stop_words)))

Tamanho do conjunto stop_words: 209 
Stop_words ordenadas: 
 ['a', 'ao', 'aos', 'aquela', 'aquelas', 'aquele', 'aqueles', 'aquilo', 'as', 'até', 'com', 'como', 'da', 'das', 'de', 'dela', 'delas', 'dele', 'deles', 'depois', 'do', 'dos', 'e', 'ela', 'elas', 'ele', 'eles', 'em', 'entre', 'era', 'eram', 'essa', 'essas', 'esse', 'esses', 'esta', 'estamos', 'estar', 'estas', 'estava', 'estavam', 'este', 'esteja', 'estejam', 'estejamos', 'estes', 'esteve', 'estive', 'estivemos', 'estiver', 'estivera', 'estiveram', 'estiverem', 'estivermos', 'estivesse', 'estivessem', 'estivéramos', 'estivéssemos', 'estou', 'está', 'estávamos', 'estão', 'eu', 'foi', 'fomos', 'for', 'fora', 'foram', 'forem', 'formos', 'fosse', 'fossem', 'fui', 'fôramos', 'fôssemos', 'haja', 'hajam', 'hajamos', 'havemos', 'haver', 'hei', 'houve', 'houvemos', 'houver', 'houvera', 'houveram', 'houverei', 'houverem', 'houveremos', 'houveria', 'houveriam', 'houvermos', 'houverá', 'houverão', 'houveríamos', 'houvesse', 'houvessem', '

In [7]:
# Aplicar o tratamento
# Usada para dividir um texto grande em partes menores (chunks) de tamanho controlado.
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Substitui valores ausentes por uma string vazia e converte todos os valores da coluna 
# "conteudo_noticia" para o tipo string
dados_esportes['conteudo_noticia'] = dados_esportes['conteudo_noticia'].fillna("").astype(str)
# Aplica a função tratamento_pln para cada valor na coluna "conteudo_noticia"
dados_esportes['conteudo_tratado'] = dados_esportes['conteudo_noticia'].apply(tratamento_pln)

# Passo 6: Dividir o texto em partes (chunks)
# Para controlar o tamanho 
# Cria um objeto text_splitter da classe RecursiveCharacterTextSplitter com os seguintes parâmetros:
# chunk_size=200 tamanho máximo dos pedaços de texto (chunks) como 200 caracteres.
# chunk_overlap=50: os pedaços de texto terão uma sobreposição de 50 caracteres entre eles.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=50)
# Cria uma lista de chunks de todos os textos na coluna "conteudo_tratado"
chunks = [
    # Itera por cada notícia (linha) na coluna conteudo_tratado
    chunk for noticia in dados_esportes['conteudo_tratado'] 
    # Para cada notícia, o split_text divide o texto em partes menores (chunks), 
    # de acordo com os parâmetros definidos anteriormente (tamanho de 200 caracteres e
    #  sobreposição de 50 caracteres)
    for chunk in text_splitter.split_text(noticia)
]


C:\Users\Daniel\AppData\Local\Temp\ipykernel_15216\549411915.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dados_esportes['conteudo_noticia'] = dados_esportes['conteudo_noticia'].fillna("").astype(str)
C:\Users\Daniel\AppData\Local\Temp\ipykernel_15216\549411915.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dados_esportes['conteudo_tratado'] = dados_esportes['conteudo_noticia'].apply(tratamento_pln)


In [ ]:
# Passo 7: Codificar os pedaços
# Biblioteca popular para gerar representações vetoriais (embeddings) de sentenças ou
# trechos de texto, converte textos em vetores numéricos que podem ser usados para tarefas
# como medir similaridade, agrupamento ou busca semântica.
from sentence_transformers import SentenceTransformer

# Cria um modelo pré-treinado para gerar embeddings de texto em português (e outras línguas) 
model = SentenceTransformer('all-MiniLM-L6-v2')
# Codifica todos os pedaços de texto (chunks) gerados anteriormente da lista "chunks" em vetores
embeddings = model.encode(chunks)


c:\Users\Daniel\OneDrive\Documentos\GitRobson\Projeto_IA_Walmir\PLN\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Passo 8: Configurar ChromaDB
# Biblioteca ara criar e gerenciar bancos de dados vetoriais, ou seja,
# armazenar e consultar embeddings de texto
# chromadb facilita a criação de coleções de dados vetoriais permitindo realizar operações,
# como consulta por similaridade.
import chromadb
# converte embeddings gerados pelo modelo SentenceTransformer para o formato adequado que o ChromaDB
#  Assim, a integração entre o modelo de embeddings e o banco de dados vetorial
#  se torna direta e simplificada.
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction

# Configurar o ChromaDB com a função de embeddings
# conecta a biblioteca chromadb ao modelo SentenceTransformer, permitindo que o ChromaDB 
# converta textos em embeddings com base no modelo all-MiniLM-L6-v2.
embedding_function = SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")
# O client é a interface principal que você usa para interagir com o ChromaDB
client = chromadb.Client()

#client.delete_collection("esportes_noticias")
# Cria uma coleção chamada "esportes_noticias" no ChromaDB
collection = client.create_collection("esportes_noticias")


In [ ]:
# Adicionar documentos ao banco de dados vetorial
# Adiciona cada pedaço de texto (chunk) ao banco de dados vetorial.
for i, chunk in enumerate(chunks):
    # Adiciona cada chunk à coleção "esportes_noticias" no ChromaDB
    collection.add(documents=[chunk], metadatas=[{'id': i}], ids=[str(i)])


In [ ]:
'''# Função para encontrar notícias semelhantes
def encontrar_similares(nova_noticia, top_k=3):
    try:
        # Codificar a nova notícia para obter seu embedding
        nova_noticia_embedding = model.encode([nova_noticia])
        
        # Consultar o ChromaDB para as notícias mais semelhantes
        resultados = collection.query(query_embeddings=nova_noticia_embedding, n_results=top_k)
        
        # Acessar os documentos e metadados
        documentos = resultados['documents']
        metadados = resultados['metadatas']
        
        return documentos, metadados
    except Exception as e:
        print(f"Erro ao buscar similares: {e}")
        return [], []'''

In [ ]:
# Função para encontrar notícias semelhantes
def encontrar_similares(texto):
    # Gerar embeddings para a nova notícia
    # texto_tratado será o texto limpo e tratado, pronto para ser transformado em um embedding.
    texto_tratado = tratamento_pln(texto)
    # Gera um embedding para o texto tratado utilizando o modelo de embeddings SentenceTransformer
    embedding_novo = model.encode([texto_tratado])
    
    # Consultar no ChromaDB
    # Realiza uma consulta no banco de dados vetorial do ChromaDB 
    resultados = collection.query(
        # Passa o embedding da nova notícia como consulta para o banco de dados.
        query_embeddings=embedding_novo,
        # Número de resultados que deseja recuperar
        n_results=5, 
    )
    
    # Contém os textos dos documentos mais semelhantes encontrados no ChromaDB
    return resultados['documents'], resultados['metadatas']



In [ ]:
# Inspeção dos resultados
print("Notícias semelhantes:")

# Chamada à função encontrar_similares
nova_noticia = "O jogador marcou um gol decisivo na final do campeonato."
encontrar_similares(nova_noticia)


Notícias semelhantes:


([['jogo série b brasileiro campeonato pernambucano campo   partida ser   titular marcar trs gol',
   'sulamericano   dois campeonato gaúcho     total   partida clube   gol marcar',
   'dois rodada final campeonato brasileiro contra goiá cruzeiro    ver tabela brasileiro',
   'lembrar penapolense vence sete jogo semifinal fazer campeonato sensacional assim fazer bom campeonato chegar semifinal conta regulamento   contra santo ir difícil santo jogar bola so Paulo   dizer',
   'semifinal campeonato carioca campeo taa guanabar cruzmaltino jogar']],
 [[{'id': 58754}, {'id': 58966}, {'id': 58110}, {'id': 2752}, {'id': 27344}]])

In [ ]:
# Passo 8: Clusterização (agrupamento)
num_clusters = 5  # Definir número de clusters
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
dados_esporte['cluster'] = kmeans.fit_predict(embeddings)



In [ ]:
# Passo 9: Geração de resumos
def gerar_resumo(texto, num_sentencas=2):
    doc = nlp(texto)
    sentencas = [sent.text for sent in doc.sents]
    return " ".join(sentencas[:num_sentencas])

dados_esportes['resumo'] = dados_esportes['conteudo_noticia'].apply(lambda x: gerar_resumo(x))



C:\Users\Daniel\AppData\Local\Temp\ipykernel_8980\1684560591.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dados_esportes['resumo'] = dados_esportes['conteudo_noticia'].apply(lambda x: gerar_resumo(x))


In [ ]:
# Contar a frequência de clusters ao longo do tempo
frequencia_clusters = dados_esporte.groupby(['ano_mes', 'cluster']).size().unstack(fill_value=0)

# Plotar tendências
frequencia_clusters.plot(kind='line', figsize=(12, 6), marker='o')
plt.title("Tendências de Tópicos ao Longo do Tempo")
plt.xlabel("Ano-Mês")
plt.ylabel("Frequência")
plt.legend(title="Cluster")
plt.grid()
plt.show()



In [ ]:
# Exibir amostra de resultados
print("\nExemplo de Resultados:")
print(dados_esporte[['data_publicacao', 'cluster', 'resumo']].head())

# Análise adicional (opcional): Principais palavras-chave por cluster
def obter_palavras_chave(cluster_id, top_n=5):
    textos = dados_esporte[dados_esporte['cluster'] == cluster_id]['conteudo_tratado']
    palavras = " ".join(textos).split()
    mais_comuns = Counter(palavras).most_common(top_n)
    return [palavra for palavra, _ in mais_comuns]

for cluster in range(num_clusters):
    palavras_chave = obter_palavras_chave(cluster)
    print(f"Cluster {cluster}: Palavras-chave - {', '.join(palavras_chave)}")
